#### Permissions
- Need to give group privilieges to schema. User is group name not user name.
`user = "voc-demo"`
- In Groups entitlement, grant Group entitlement for Workspace (default to not)
- Need to grant permissions to SP for SQL endpoint


- Permissions to UC, Genie Room to All SP

- Permission to Compute clusters (sql)

## Set up Service Principal & Row level filter

#### Example Setup, Replace

In [0]:
%sql
-- Step 1: Create the mapping table
CREATE OR REPLACE TABLE lakehouse_inn_catalog.voc.sp_location_acl (
  service_principal_name STRING,
  service_principal_id STRING,
  allowed_location STRING
);
--  TODO: replace sample service principals and users
INSERT INTO lakehouse_inn_catalog.voc.sp_location_acl VALUES
  ('boston-demo-sp', 'c29e43e6-d791-4314-bdea-87440b7e0212', 'Boston, MA'),
  ('austin-demo-sp', '067df0c1-f8b9-40e7-a195-86f23251a24d','Austin, TX');

-- Step 2: Create the row filter function
--  TODO: replace with acutal users

CREATE OR REPLACE FUNCTION lakehouse_inn_catalog.voc.location_acl_filter(
  location STRING
)
RETURNS BOOLEAN
RETURN
  current_user() IN ('cindy.wu@databricks.com', 'robert.mosley@databricks.com', 'dan.morris@databricks.com', 'krishna.r@databricks.com
', 'bdae5339-4ad3-4c8d-b45d-02d2134752b8', '4bba4b51-48c5-4331-964b-db6c6a35f06e')
  OR EXISTS (
    SELECT 1
    FROM lakehouse_inn_catalog.voc.sp_location_acl
    WHERE service_principal_id = current_user()
      AND allowed_location = location
  )
;

In [0]:
%sql
ALTER TABLE lakehouse_inn_catalog.voc.issues_daily
SET ROW FILTER lakehouse_inn_catalog.voc.location_acl_filter ON (location);

In [0]:
%sql
ALTER TABLE lakehouse_inn_catalog.voc.raw_reviews
SET ROW FILTER lakehouse_inn_catalog.voc.location_acl_filter ON (location);

In [0]:
%sql
ALTER TABLE lakehouse_inn_catalog.voc.review_aspect_details
SET ROW FILTER lakehouse_inn_catalog.voc.location_acl_filter ON (location);

## Test out row filters

In [0]:
%pip install databricks-sql-connector
%restart_python


In [0]:
import os
os.environ["SP_DATABRICKS_CLIENT_SECRET"] =''
os.environ["SP_DATABRICKS_CLIENT_ID"] ='c29e43e6-d791-4314-bdea-87440b7e0212'
os.environ["DATABRICKS_HTTP_PATH"] = "/sql/1.0/warehouses/a9fea331792bc9d6"
os.environ["DATABRICKS_SERVER_HOSTNAME"] = 'fe-vm-voc-lakehouse-inn-workspace.cloud.databricks.com'

In [0]:
from databricks import sql
from databricks.sdk.core import Config

from databricks.sdk.core import Config, oauth_service_principal
from databricks import sql
import os

server_hostname = os.getenv("DATABRICKS_SERVER_HOSTNAME")
def credential_provider():
  config = Config(
    host          = f"https://{server_hostname}",
    client_id=os.getenv("SP_DATABRICKS_CLIENT_ID"),
    client_secret=os.getenv("SP_DATABRICKS_CLIENT_SECRET"))
  return oauth_service_principal(config)

with sql.connect(server_hostname      = server_hostname,
                 http_path            = os.getenv("DATABRICKS_HTTP_PATH"),
                 credentials_provider = credential_provider) as conn:
    query = "SELECT * FROM lakehouse_inn_catalog.voc.issues_daily"
    # query = "SELECT * FROM lakehouse_inn_catalog.voc.review_aspect_details"

    # query = "SELECT current_user();"
    with conn.cursor() as cursor:
        cursor.execute(query)
        df = cursor.fetchall_arrow().to_pandas()
        print(df.head())
    conn.close()

## (Optional or UI) Sync runbook table to Lakebase (for app performance)


In [0]:
# from databricks.sdk import WorkspaceClient
# from databricks.sdk import WorkspaceClient
# from databricks.sdk.service.database import SyncedDatabaseTable, SyncedTableSpec, NewPipelineSpec, SyncedTableSchedulingPolicy
# # Initialize the Workspace client
# w = WorkspaceClient()

# # Create a synced table in a database catalog
# synced_table = w.database.create_synced_database_table(
#     SyncedDatabaseTable(
#         name="database_catalog.schema.synced_table",  # Full three-part name
#         spec=SyncedTableSpec(
#             source_table_full_name="source_catalog.source_schema.source_table",
#             primary_key_columns=["id"],  # Primary key columns
#             scheduling_policy=SyncedTableSchedulingPolicy.TRIGGERED,  # SNAPSHOT, TRIGGERED, or CONTINUOUS
#             # Optional: timeseries_key="timestamp"  # For deduplication
#             new_pipeline_spec=NewPipelineSpec(
#                 storage_catalog="storage_catalog",
#                 storage_schema="storage_schema"
#             )
#         ),
#     )
# )
# print(f"Created synced table: {synced_table.name}")
